In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir -p /content/drive/MyDrive/hf-demo

In [3]:
%cd /content/drive/MyDrive/hf-demo

/content/drive/MyDrive/hf-demo


In [4]:
# CUDA 12.1 destekli PyTorch, torchvision ve torchaudio kurulumu
!pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Hugging Face ekosistemi
!pip install --quiet transformers accelerate

# Gradio (demo arayüzler için)
!pip install --quiet gradio

In [5]:
import torch

print("Torch versiyonu:", torch.__version__)
print("CUDA kullanılabilir mi?:", torch.cuda.is_available())
print("CUDA versiyonu:", torch.version.cuda)
print("GPU ismi:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Yok")

Torch versiyonu: 2.8.0+cu126
CUDA kullanılabilir mi?: True
CUDA versiyonu: 12.6
GPU ismi: Tesla T4


In [6]:
import transformers
transformers.__version__

'4.55.4'

In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
from transformers import pipeline

In [9]:
generator = pipeline("text-generation",
                     model = "microsoft/Phi-4-mini-instruct",
                     torch_dtype ="auto",
                     device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
messages = [
  {"role": "system", "content": "You are a helpful and curious AI assistant that helps users reflect on their personal financial habits. You ask thought-provoking and insightful questions to better understand them."},
  {"role": "user", "content": "Please ask me three smart questions about my financial behavior to help me reflect and improve."}
]


In [11]:
outputs = generator(
    messages,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    return_full_text = False

)

In [12]:
print(outputs[0]["generated_text"])

1. How do you currently approach saving and budgeting, and what goals are you aiming to achieve with your financial planning?
2. Can you identify any recurring patterns in your spending habits that might lead to overspending or under-saving in certain areas?
3. What strategies do you use to prioritize your financial goals, especially when faced with unexpected expenses or financial setbacks?


In [13]:
import gradio as gr

In [17]:
def generate_text(prompt):
  messages = [
    {"role": "system", "content": "You are a helpful and curious AI assistant that helps users reflect on their personal financial habits. You ask thought-provoking and insightful questions to better understand them."},
    {"role": "user", "content": prompt}
  ]
  outputs = generator(
    messages,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.7,
    return_full_text = False
  )
  return outputs[0]["generated_text"]

In [18]:
demo=gr.Interface(
    fn=generate_text,
    inputs = gr.Textbox(label='Enter a prompt!'),
    outputs = gr.Textbox(label='Output'),
    title = "Financial AI Chatbot"
)

In [19]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5f56f5dd9617ea448.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
